In [ ]:
#pip install --upgrade azureml-core

In [ ]:
#pip install --upgrade raiwidgets

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.43.0


In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [3]:
import azureml.core
from azureml.core import Workspace
import pandas as pd

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.43.0 to work with mlopswk


In [4]:
df= pd.read_csv('./Data/titanic.csv')
print(df.shape)
print(df.columns)

(891, 12)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [5]:
df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
df['Loc']= df['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'X')

In [8]:
df.drop(['Cabin', 'Ticket'], axis=1, inplace=True)

In [9]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Loc
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,X
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,X
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,X


In [10]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
Loc            0
dtype: int64

In [11]:
df.loc[:,'GroupSize'] = 1 + df['SibSp'] + df['Parch']

In [12]:
df['Embarked'] = df['Embarked'].fillna('S')

In [13]:
df['Sex'] = df['Sex'].apply({'male':1, 'female':2}.get)

In [14]:
df['Embarked'] = df['Embarked'].apply({'C':1, 'Q':2, 'S':3}.get)

In [15]:
df['Loc'] = df['Loc'].apply({'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'T':8, 'X':9}.get)

In [16]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Loc,GroupSize
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,7.2500,3,9,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",2,38.0,1,0,71.2833,1,3,2
2,3,1,3,"Heikkinen, Miss. Laina",2,26.0,0,0,7.9250,3,9,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",2,35.0,1,0,53.1000,3,3,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,8.0500,3,9,1


In [17]:
LABEL = 'Survived'
columns_to_keep = ['Pclass', 'Sex','Age', 'Fare', 'Embared', 'Deck', 'GroupSize']
columns_to_drop = ['Name','SibSp', 'Parch', 'Survived']
df_train = df
df = df_train.drop(['Name','SibSp', 'Parch', 'PassengerId'], axis=1)

df.head(5)

,Survived,Pclass,Sex,Age,Fare,Embarked,Loc,GroupSize
0,0,3,1,22.0,7.2500,3,9,2
1,1,1,2,38.0,71.2833,1,3,2
2,1,3,2,26.0,7.9250,3,9,1
3,1,1,2,35.0,53.1000,3,3,2
4,0,3,1,35.0,8.0500,3,9,1


In [18]:
df1 = pd.get_dummies(df)

In [19]:
df1.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Loc,GroupSize
0,0,3,1,22.0,7.2500,3,9,2
1,1,1,2,38.0,71.2833,1,3,2
2,1,3,2,26.0,7.9250,3,9,1
3,1,1,2,35.0,53.1000,3,3,2
4,0,3,1,35.0,8.0500,3,9,1


In [20]:
import os
script_folder = os.path.join(os.getcwd(), "train_remote")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox2022/code/Users/babal/AMLBook2022/AMLBook2022/Chapter3/train_remote


In [21]:
import shap
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [22]:
df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Loc,GroupSize
0,0,3,1,22.0,7.2500,3,9,2
1,1,1,2,38.0,71.2833,1,3,2
2,1,3,2,26.0,7.9250,3,9,1
3,1,1,2,35.0,53.1000,3,3,2
4,0,3,1,35.0,8.0500,3,9,1


In [23]:
y = df['Survived']
df = df.drop(['Survived'], axis=1)

In [24]:
df = pd.get_dummies(df)

In [25]:
df.head()

,Pclass,Sex,Age,Fare,Embarked,Loc,GroupSize
0,3,1,22.0,7.2500,3,9,2
1,1,2,38.0,71.2833,1,3,2
2,3,2,26.0,7.9250,3,9,1
3,1,2,35.0,53.1000,3,3,2
4,3,1,35.0,8.0500,3,9,1


In [26]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [27]:
target_feature = 'Survived'

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=7)

train_data = X_train.copy()
test_data = X_test.copy()
train_data[target_feature] = y_train
test_data[target_feature] = y_test
#data.feature_names

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [30]:
#model = RandomForestRegressor()
model = RandomForestClassifier(max_depth=2, random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [31]:
pred = model.predict(X_test)

In [32]:
pred_proba = model.predict_proba(X_test)

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [34]:
# View accuracy score
accuracy_score(y_test, pred)

0.7821229050279329

In [35]:
# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred)

array([[104,   6],
       [ 33,  36]])

In [36]:
# View the classification report for test data and predictions
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.76      0.95      0.84       110
           1       0.86      0.52      0.65        69

    accuracy                           0.78       179
   macro avg       0.81      0.73      0.75       179
weighted avg       0.80      0.78      0.77       179



In [37]:
#pred_proba

In [38]:
train_data.dtypes

Pclass         int64
Sex            int64
Age          float64
Fare         float64
Embarked       int64
Loc            int64
GroupSize      int64
Survived       int64
dtype: object

In [39]:
test_data.dtypes

Pclass         int64
Sex            int64
Age          float64
Fare         float64
Embarked       int64
Loc            int64
GroupSize      int64
Survived       int64
dtype: object

In [40]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

In [41]:
train_data.count()

Pclass       712
Sex          712
Age          712
Fare         712
Embarked     712
Loc          712
GroupSize    712
Survived     712
dtype: int64

In [42]:
rai_insights = RAIInsights(model, train_data[:20], test_data[:20], target_feature, 'classification',
                               categorical_features=[])

In [43]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()
# Counterfactuals: accepts total number of counterfactuals to generate, the range that their label should fall under, 
# and a list of strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=10, desired_class='opposite')

In [44]:
rai_insights.compute()

100%|██████████| 20/20 [03:13<00:00,  9.69s/it]


In [45]:
ResponsibleAIDashboard(rai_insights)

ResponsibleAI started at https://devbox2022-5000.centralus.instances.azureml.ms


In [ ]:
#pip install pandas-profiling[notebook]

In [46]:
from pandas_profiling import ProfileReport

In [49]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)

In [50]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]